In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [107]:
class CharCNN(nn.Module):
    
    def __init__(self, n_classes, num_chars=70, max_seq_length=1014, kernel_sizes= [7,7,3,3,3,3], 
                 channel_size=256, pool_size=3):
        
        super(CharCNN, self).__init__()
        
        self.num_chars = num_chars
        self.max_seq_length = max_seq_length
        self.n_classes = n_classes
        self.pool_size = pool_size
        self.final_linear_len = (max_seq_length - 96)/ 27
        
        self.conv1 = nn.Conv1d(num_chars, channel_size, kernel_sizes[0])
        self.conv2 = nn.Conv1d(channel_size, channel_size, kernel_sizes[1])
        self.conv3 = nn.Conv1d(channel_size, channel_size, kernel_sizes[2])
        self.conv4 = nn.Conv1d(channel_size, channel_size, kernel_sizes[3])
        self.conv5 = nn.Conv1d(channel_size, channel_size, kernel_sizes[4])
        self.conv6 = nn.Conv1d(channel_size, channel_size, kernel_sizes[5])
        
        self.pool1 = nn.MaxPool1d(pool_size)
        self.pool2 = nn.MaxPool1d(pool_size)
        self.pool3 = nn.MaxPool1d(pool_size)
        
        self.FC1 = nn.Linear(channel_size * self.final_linear_len, 1024)
        self.FC2 = nn.Linear(1024, 1024)
        self.final_layer = nn.Linear(1024, n_classes)
        
        self.dropout1 = nn.Dropout(p=0.5)
        self.dropout2 = nn.Dropout(p=0.5)
        
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                m.weight.data.normal_(0, 0.05)
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.05)

        
    def forward(self, x):        
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool3(F.relu(self.conv6(x)))
        x = x.view(x.size(0),-1)
        x = self.FC1(x)
        x = self.dropout1(x)
        x = self.FC2(x)
        x = self.dropout2(x)
        x = self.final_layer(x)
        return F.log_softmax(x)     

In [108]:
charcnn = CharCNN(10)

In [109]:
t = Variable(torch.randn(64,70,1014))

Testing the forward pass with a random input with batch size 64

In [110]:
charcnn(t)

Variable containing:

Columns 0 to 7 
 -68.8173 -104.9899  -25.6022  -42.4737  -80.5405   -0.0011  -27.8097   -6.8594
  -0.0000  -50.6404 -154.2055 -105.0202  -62.1325  -54.2097  -39.9622  -66.6271
   0.0000 -206.8687 -120.9110 -179.8722 -103.0993  -99.9894 -118.5074 -220.0174
-157.6653 -149.9798    0.0000 -221.4961 -100.3018 -165.9884 -176.3581 -107.7630
 -88.3535  -28.9064 -125.5320 -158.0676 -212.1708  -51.4926  -39.0871 -130.1813
-105.3856  -15.5726  -39.3309 -199.9827 -194.3597  -26.0501   -0.0013 -149.4773
  -0.0000  -66.4504  -52.4241 -103.8333 -116.5250  -18.8425  -65.4385 -135.8221
 -29.3479 -120.5390  -97.0079 -122.2001  -43.9130 -106.3288   -0.0000  -75.2879
 -98.2197  -36.9157 -151.0684  -96.4283 -135.4412   -0.0001 -168.7530 -123.6483
 -30.2257 -111.6708  -35.5697  -57.5128 -168.4377   -0.0000 -101.8533  -84.4177
   0.0000 -201.5231 -129.1893 -180.1489 -181.6742 -201.0459 -197.9904 -264.5238
 -67.5501 -245.7548    0.0000 -181.5725 -145.6481  -92.1788 -105.0201 -217.6792
  